In [1]:
import pyvisa                                     # Required to talk to Keithley
import serial                                   # Required to connect to USB Devices
import msvcrt                                   # required to detect keyboard press
import time                                     # Required to sleep
import os                                       # Required to change directory
import datetime                                 # Required to figure out current system time
import math                                     # Required to do complex math
import csv                                      # Required to save to .csv files
import numpy as np                              # Required for arrays 
import pandas as pd                             # Required for Data Frames
import statistics as st

from tkinter import *                           #
from tkinter.filedialog import askopenfilename  # Required for choosing file location
from tkinter.filedialog import askdirectory     #
from matplotlib import pyplot as plt            # Required for plots
import matplotlib.ticker as ticker              # Required for grid on plots
import datetime                                 # Required to log time of errors
#%matplotlib

In [2]:
## Select Number of Events to be saved
NumEvents=300


In [3]:
print("\n")
print("#############################################################")
print("###                                                       ###")
print("###    Press ENTER to choose the folder for where the     ###")
print("###            lifetime_test current measurement          ###")
print("###                 data will be stored.                  ###")
print("###                                                       ###")
print("#############################################################")
file = input("")            # Waits for user to press enter before File 
foldername = askdirectory() # Opens File Explorer to choose and save file location
os.chdir(foldername)        # Changes the directory to chosen file location
Tk().destroy()             # Closes the main Tk wiindow



#############################################################
###                                                       ###
###    Press ENTER to choose the folder for where the     ###
###            lifetime_test current measurement          ###
###                 data will be stored.                  ###
###                                                       ###
#############################################################



In [4]:
rm=pyvisa.ResourceManager()
rm.list_resources()

('TCPIP0::129.107.83.133::inst0::INSTR',
 'ASRL1::INSTR',
 'ASRL2::INSTR',
 'ASRL4::INSTR',
 'ASRL6::INSTR',
 'ASRL7::INSTR',
 'ASRL8::INSTR')

In [5]:
my_instrument = rm.open_resource('ASRL7::INSTR',read_termination='\r')
print(my_instrument.query("*IDN?"))
time.sleep(5)



KEITHLEY INSTRUMENTS INC.,MODEL 6485,4125305,C01   Jun 23 2010 12:22:00/A02  /J


In [6]:
time.sleep(5)
my_instrument.write("TRAC:CLE")
my_instrument.write("ARM:SOUR IMM")
my_instrument.write("ARM:COUN 1")
my_instrument.write("TRIG:SOUR IMM")
my_instrument.write("TRIG:COUN 1")
Sum = 0
k = 0
file_num = 0
Start = float(my_instrument.query("READ?").split(",")[1].replace("+", ""))
print("Press Enter in the command prompt to exit the data acquisition")
Time = []
Current = []

for i in range(0,NumEvents):
    my_instrument.write("TRAC:CLE")
    X = my_instrument.query("READ?")
    curr=float(X.split(",")[0].replace("+", "").replace("A",""))
    tim=float(X.split(",")[1].replace("+", ""))

    if((curr < 1.0*10**-11) or (curr > 2.0*10**-11)):
        Log = open("Error_Log.txt", "a")
        Log.write("{} {}\n".format(curr,datetime.datetime.now))
        Log.close()

    Current.append(curr)
    Time.append(tim - Start)
    time.sleep(1)
## Plotting and saving the current values
plt.figure(figsize = (12,8))
plt.xlim(Time[0],Time[len(Time)-1]+2.5)
plt.plot(Time, Current)
plt.savefig('Keithley_Time_Current_{}.png'.format(file_num))
plt.close()
Save = np.array(pd.concat([pd.DataFrame(Current), pd.DataFrame(Time)], axis=1))
np.savetxt('current_time{}.txt'.format(file_num), Save)
Avg_log = open("Current Avg Log.txt", "w")
Avg_log.write("Current(A) Stdev(A) Date \n")

Avg_log.write("{} {} {}\n".format(st.mean(Current),st.stdev(Current), datetime.datetime.now()))

Avg_log.close()    
file_num = file_num + 1
time.sleep(2)
rm.close()

Press Enter in the command prompt to exit the data acquisition
